## Playground, agents and behaviors definitions

In [ ]:
from simulator_interface import open_session, close_session
import random as rd
import math

agents = [
    { 
        "agent_type": "predator",
        "initial_coordinates": ((0.5,0.6),2),
        "radius": 20,
        "texture": (192, 106, 47)
    },
    { 
        "agent_type": "prey",
        "initial_coordinates": ((0.5,0.4), 2*rd.random()*math.pi),
        "radius": 15,
    },
    { 
        "agent_type": "prey",
        "initial_coordinates": ((0.4,0.2), 2*rd.random()*math.pi),
        "radius": 15,
    },
    { 
        "agent_type": "prey",
        "initial_coordinates": ((0.6,0.2), 2*rd.random()*math.pi),
        "radius": 15,
    }
]

playground = {
    "size": (600,600),
    "interactions": ["BIG_EAT_SMALL"]
}


def obstacle_avoidance(robot):
    left, right = robot.prox_activations(tracked_objects=['Wall'])
    return 1 - right, 1 - left
    
def aggression(robot):
    left, right = robot.prox_activations(tracked_objects=["prey"])
    return right, left

def fear(robot):
    left, right = robot.prox_activations(tracked_objects=["predator"])
    return left, right

def seek_candies(robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])
    return right, left

## Start simulator

In [ ]:
simulation = open_session(nb_agents=len(agents), agents=agents, playground_params=playground)

simulator = simulation[0]
predator = simulation[1]
preys = simulation[2:]

In [ ]:
simulator.start_sphere_apparition(period=10, eatable=True, radius=8)

## Attach and start behaviors

In [ ]:
for prey in preys:
    prey.attach_behavior(obstacle_avoidance, freq=1)
    prey.attach_behavior(fear, freq=3)
    prey.attach_behavior(seek_candies, freq=5)
    prey.start_behavior(seek_candies)
    prey.start_behavior(fear)
    prey.start_behavior(obstacle_avoidance)

predator.attach_behavior(obstacle_avoidance, freq=1)
predator.attach_behavior(aggression, freq=5)
predator.start_behavior(aggression)
predator.start_behavior(obstacle_avoidance)

## Stop behaviors

In [ ]:
simulator.stop_sphere_apparition()

for prey in preys:
    prey.stop_behavior(fear)
    prey.stop_behavior(seek_candies)
    prey.stop_behavior(obstacle_avoidance)
    prey.stop()

predator.stop_behavior(aggression)
predator.stop_behavior(obstacle_avoidance)
predator.stop()

## Detach behaviors

In [ ]:
for prey in preys:
    prey.detach_all_behaviors()

predator.detach_all_behaviors()

## Close simulator

In [ ]:
close_session(simulator)